#### 2.3.5 _Programming Exercise:_ Generate a 2-of-2 `csahasholder` tapscript

In [ ]:
# Generate key pairs
privkey0 = ECKey().generate()
privkey1 = ECKey().generate()

pubkey0 = privkey0.get_pubkey()
pubkey1 = privkey1.get_pubkey()

print("pubkey0: {}".format(pubkey0.get_bytes().hex()))
print("pubkey1: {}\n".format(pubkey1.get_bytes().hex()))

# Method: 32B preimage - sha256(bytes)
# Method: 20B digest - hashlib.new('ripemd160', bytes).digest()
secret = b'secret'
preimage = sha256(secret)
digest = hashlib.new('ripemd160', preimage).digest()
delay = 20

# Construct Tapscript
csahasholder_tapscript = TapLeaf()
csahasholder_tapscript.construct_csahasholder(2, [pubkey0, pubkey1], digest, delay)
print("Descriptor:", csahasholder_tapscript.desc, "\n")

print("Tapscript operations:")
for op in csahasholder_tapscript.script:
    print(op.hex()) if isinstance(op, bytes) else print(op)

print("\nSatisfying witness elements:")
for element, value in csahasholder_tapscript.sat:
    print("{}, {}".format(element, value.hex()))

#### _2.3.6 Programming Exercise:_ Compute the taptweak from a tapscript

In [ ]:
def tagged_hash(tag, input_data):
    data = sha256(tag.encode('utf-8'))
    data += data
    data += input_data
    return sha256(data)

privkey_internal = ECKey().generate()
pubkey_internal = privkey_internal.get_pubkey()

# Method: ser_string(Cscript) prepends compact size.
TAPSCRIPT_VER = bytes([0xc0])
tapleaf = tagged_hash("TapLeaf", TAPSCRIPT_VER + ser_string(csahasholder_tapscript.script))
taptweak = tagged_hash("TapTweak", pubkey_internal.get_bytes() + tapleaf)
print("Your constructed taptweak is: {}.".format(taptweak.hex()))

#### _Programming Exercise 2.3.12:_ Construct `CTransaction` and populate inputs

In [ ]:
# Construct transaction
spending_tx = CTransaction()

# Populate the transaction version
spending_tx.nVersion = 2

# Populate the locktime
spending_tx.nLockTime = 0

# Populate the transaction inputs
# Method: Construct COutPoint(txid, index)
# Method: Construct CTxIn(outpoint = ..., nSequence = ...)
# Tip: CTransaction.vin = "list of CTxIn objects"
outpoint = COutPoint(tx.sha256, output_index)
spending_tx_in = CTxIn(outpoint = outpoint, nSequence=delay)
spending_tx.vin = [spending_tx_in]

print("Spending transaction:\n{}".format(spending_tx))

#### _Programming Exercise 2.3.14:_ Sign the transaction

In [ ]:
# Generate the Taproot Signature Hash for signing
sighash = TaprootSignatureHash(spending_tx, [output], SIGHASH_ALL_TAPROOT, input_index = 0, scriptpath = True, tapscript=csahasholder_tapscript.script)

# Sign with both privkeys
signature0 = privkey0.sign_schnorr(sighash)
signature1 = privkey1.sign_schnorr(sighash)

print("Signature0: {}".format(signature0.hex()))
print("Signature1: {}".format(signature1.hex()))

#### _Programming Exercise 2.3.15:_ Add the witness and test acceptance of the transaction

In [ ]:
# Construct transaction witness
# Tip: Witness stack for script path - [satisfying elements for tapscript] [TapLeaf.script] [controlblock]
# Tip: Controlblock for a tapscript in control_map[TapLeaf.script]
witness = CScriptWitness()
witness.stack = [preimage, signature1, signature0, csahasholder_tapscript.script, control_map[csahasholder_tapscript.script]]
witness_in = CTxInWitness()
witness_in.scriptWitness = witness

# vtxinwit is a list of the witness data(i.e. signatures etc.)
spending_tx.wit.vtxinwit.append(witness_in)

print("Spending transaction:\n{}\n".format(spending_tx))

# Serialize signed transaction for broadcast
spending_tx_str = spending_tx.serialize().hex()

# Test mempool acceptance with and without delay.
assert not test.nodes[0].testmempoolaccept([spending_tx_str])[0]['allowed']
test.nodes[0].generate(delay)
assert test.nodes[0].testmempoolaccept([spending_tx_str])[0]['allowed']

print("Success!")